In [1]:
import numpy as np
import re
from pyspark.sql import SparkSession
from nltk.tokenize import sent_tokenize
from pyspark.sql.functions import udf
from pyspark.sql.functions import lit
from pyspark.sql.types import *
from pyspark.sql import functions as F
from pyspark.ml.feature import Word2Vec
from pyspark.sql.functions import split, explode
from pyspark.sql.types import ArrayType, StringType

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
0,application_1558522583055_0001,pyspark,idle,Link,Link,✔


SparkSession available as 'spark'.


In [2]:
spark = SparkSession \
    .builder \
    .appName("Stage 4") \
    .getOrCreate()

VBox()

In [3]:
rev_data = "s3://amazon-reviews-pds/tsv/amazon_reviews_us_Music_v1_00.tsv.gz"
revs = spark.read.csv(rev_data,header=True,sep='\t')

VBox()

In [4]:
selectedproduct = revs.filter(revs.product_id=='B004EBT5CU')

VBox()

In [5]:
productRecBefore = selectedproduct.select("review_id", "review_body", "star_rating")

VBox()

In [6]:
productRec = productRecBefore.filter(productRecBefore.review_body != '')

VBox()

In [7]:
def filterout_htmltag(x):
    tag_pattern = re.compile('<.*?>')
    cleantag = re.sub(tag_pattern, ' ', x)
    return cleantag

clean_htmltag = udf(lambda x: filterout_htmltag(x), StringType())

productRec = productRec.withColumn("review_body", lit(clean_htmltag(productRec['review_body'])))

VBox()

In [10]:
pattern = re.compile(r'[\.\?\!]+ ')
def segmentReview(x):
    contentList = re.split(pattern, x)
    return contentList

contentOfSen = udf(lambda x: segmentReview(x), ArrayType(StringType()))

VBox()

In [11]:
def segmentReviewnlp(x):
    contentlist = sent_tokenize(x)
    return contentlist

contentofsennlp = udf(lambda x: segmentReviewnlp(x), ArrayType(StringType()))

VBox()

In [16]:
posClass = productRec.filter(productRec.star_rating >= 4).withColumn("sentences", contentOfSen(productRec.review_body)).cache()
negClass = productRec.filter(productRec.star_rating <= 2).withColumn("sentences", contentOfSen(productRec.review_body)).cache()
posClassSen = posClass.withColumn('sentence', explode(posClass.sentences))
negClassSen = negClass.withColumn('sentence', explode(negClass.sentences))

VBox()

In [17]:
def stringtolist(sentence):
    pattern = re.compile(r'\s+|[\.\?\!]+ ')
    sentencelist = re.split(pattern, sentence)
    return sentencelist
newList = udf(lambda x: stringtolist(x), ArrayType(StringType()))
posSentence = posClassSen.select('sentence','review_id').withColumn('sentencesplit',newList(posClassSen.sentence))
negSentence = negClassSen.select('sentence','review_id').withColumn('sentencesplit',newList(negClassSen.sentence))

VBox()

# Positive reviews

In [18]:
word2Vec = Word2Vec(vectorSize=512, minCount=0, inputCol="sentencesplit", outputCol="result")
model = word2Vec.fit(posSentence)
posResultTrans = model.transform(posSentence)
posResult = posResultTrans.collect()

VBox()

----------------------------------------
Exception happened during processing of request from ('127.0.0.1', 54656)
----------------------------------------
Traceback (most recent call last):
  File "/usr/lib64/python2.7/SocketServer.py", line 293, in _handle_request_noblock
    self.process_request(request, client_address)
  File "/usr/lib64/python2.7/SocketServer.py", line 321, in process_request
    self.finish_request(request, client_address)
  File "/usr/lib64/python2.7/SocketServer.py", line 334, in finish_request
    self.RequestHandlerClass(request, client_address, self)
  File "/usr/lib64/python2.7/SocketServer.py", line 655, in __init__
    self.handle()
  File "/usr/lib/spark/python/lib/pyspark.zip/pyspark/accumulators.py", line 266, in handle
    poll(authenticate_and_accum_updates)
  File "/usr/lib/spark/python/lib/pyspark.zip/pyspark/accumulators.py", line 241, in poll
    if func():
  File "/usr/lib/spark/python/lib/pyspark.zip/pyspark/accumulators.py", line 254, in authe

In [31]:
def putvectorsintolist(result):
    thelist = []
    for row in result:
        thelist.append(np.array(row.result))
    return thelist
posVectorList = putvectorsintolist(posResult)

VBox()

In [33]:
def calCosine(vector1, vector2):
    result = np.dot(vector1,vector2)
    norm = np.linalg.norm(vector1)*np.linalg.norm(vector2)
    cos = result/norm
    return (1-cos)

VBox()

In [34]:
def calAverage(list):
    list2 = []
    for x in list:
        list1 = []
        for y in list:
            if ((x == y).all()) == False:
                list1.append(calCosine(x, y))
        average = sum(list1)/len(list1)
        list2.append(average)
    return list2

VBox()

In [35]:
posResultAve = calAverage(posVectorList)
posVectorList = putvectorsintolist(posResult)

VBox()

In [38]:
def putvectorsintolist(result):
    thelist = []
    for row in result:
        thelist.append(np.array(row.result))
    return thelist

VBox()

In [39]:
posMinAveIndex = posResultAve.index(min(posResultAve))

VBox()

In [40]:
def calculate_distance(vector_list,point_index):
    vector = vector_list[point_index]
    dic = {}
    for i in range(len(vector_list)):
        othervector = vector_list[i]
        if (vector == othervector).all() == False:
            dic[i]= calCosine(vector, othervector)

    return dic
posDistance_list = calculate_distance(posVectorList,posMinAveIndex)

VBox()

In [41]:
x = sorted(posDistance_list.items(),key=lambda item:item[1])
posToptenList = x[:10]

VBox()

In [42]:
posKeyList = []
for i in posToptenList:
    posKeyList.append(i[0])

VBox()

In [43]:
def getrows(df, rownums=None):
    return df.rdd.zipWithIndex().filter(lambda x: x[1] in rownums).map(lambda x: x[0])

def gettext(result,rownum):
    therow = getrows(result, rownums=[rownum]).collect()
    review_id = therow[0].review_id
    sentence_text = therow[0].sentence

    return [str(review_id), str(sentence_text)]

VBox()

In [44]:
outputListPos = gettext(posResultTrans,posMinAveIndex)

VBox()

In [45]:
def ten_close_neighbor(result,indexlist):
    tempList = []
    for i in indexlist:
        tempList.extend(gettext(result,i))
        
    return tempList
    
nListPos = ten_close_neighbor(posResultTrans,posKeyList)
outputListPos.extend(nListPos)

VBox()

In [46]:
count = 0
for i in range(len(outputListPos)):
    if count == 0:
        print("Center Sentence:")
        count += 1
    if count == 2:
        print("Top 10 Neighbor:")
        count += 1
    if i%2 == 0:
        print("review_id: " + outputListPos[i])
    else:
        print("sentence: " + outputListPos[i])
        print
        count += 1

VBox()

Center Sentence:
review_id: R3GKI20L66NJJG
sentence:  This is an album that you buy, insert into your cd/mp3 player and let it play because you're not going to need to skip over any of the songs since each track is ALL THAT

Top 10 Neighbor:
review_id: R3LT7TCVAS6NPB
sentence: It is nice to see new young people coming into the music industry that are actually worth listening to

review_id: RXQ6T10WQX1F1
sentence: It has much better songs and anyone who has gone through a relationship either good or bad will be able to relate to the songs on this album.

review_id: RTRQYVWDS6O9L
sentence: Her vocal styling is showcased as always and she gives us I do enjoy it when she incorporates her own interpretations of well known songs like she did before with Dylan's \\"To Make You Feel My Love\\" and this time around it is \\"Lovesong\\" I appreciate artists writing what they know and at 21 Adele knows a thing or two about heartbreak and regret but not in an overly melancholic way

review_id: R2B

# Negtive reviews

In [47]:
word2Vec = Word2Vec(vectorSize=512, minCount=0, inputCol="sentencesplit", outputCol="result")
model = word2Vec.fit(negSentence)
negResultTrans = model.transform(negSentence)
negResult = negResultTrans.collect()

VBox()

In [48]:
negVectorList = putvectorsintolist(negResult)
negResultAve = calAverage(negVectorList)

VBox()

In [49]:
negMinAveIndex = negResultAve.index(min(negResultAve))

VBox()

In [50]:
negDistance_list = calculate_distance(negVectorList,negMinAveIndex)

VBox()

In [51]:
x = sorted(negDistance_list.items(),key=lambda item:item[1])
negToptenList = x[:10]

VBox()

In [52]:
negKeyList = []
for i in negToptenList:
    negKeyList.append(i[0])

VBox()

In [55]:
outputListNeg = gettext(negResultTrans,negMinAveIndex)

VBox()

In [56]:
nListNeg = ten_close_neighbor(negResultTrans,negKeyList)
outputListNeg.extend(nListNeg)

VBox()

In [57]:
count = 0
for i in range(len(outputListNeg)):
    if count == 0:
        print("Center Sentence:")
        count += 1
    if count == 2:
        print("Top 10 Neighbor:")
        count += 1
    if i%2 == 0:
        print("review_id: " + outputListNeg[i])
    else:
        print("sentence: " + outputListNeg[i])
        print
        count += 1

VBox()

Center Sentence:
review_id: R2XV9CUVITLZKZ
sentence: Adele is a very talented artist and I couldn't wait to hear the CD on a high resolution audio system

Top 10 Neighbor:
review_id: R14FYAKHBG3YZL
sentence: Adele is a fine fresh artist with a powerful voice and the songs on this album are musically wonderful

review_id: RLGSI7PPKZLEY
sentence: I know what is in store for me and I thank Virgin Fm which is piped through the speakers here at work for drumming it into my head that this woman simply is unable to sing to save her life

review_id: R1N25DVLK6N2SO
sentence: I purchased a Adele 21 CD and I am quite disappointed that the CD has skips on it

review_id: R19DG79K9WW0FN
sentence: Rolling in the Deep is a great song and there is no disputing Adele has a wonderful voice

review_id: R3VG1SV0OJNUX2
sentence: I suppose that in this day and age of women that can't get record deals without exploiting their physical appearance, I should be glad that Adele is the first artist in a long time 